# Evaluation of different crowd analysis metrics


(1) Dataset filtering:

* Exclude workers based on:
    - contradiction ration (absolut thresh, batch stdv, pair stdv, total stdv)
    - worker quality score (thresholds)
    - attention check fails 
    
(2) Label aggregation:

* Majority vote
* Top vote
* CT unit-label score (tresholds)

In [2]:
from evaluation import evaluate 
from aggregation import aggregate_binary_labels
from load_data import load_expert_data
from load_data import load_experiment_data
from clean_annotations import clean_worker_cont_rate
from calculate_iaa import get_agreement 

import pandas as pd

## Load data

### Load expert data

In [6]:
# load expert data
run = 4
group = 'expert_inspection*'
n_q = '*'
batch = '*'
data_dict_list_exp = load_expert_data(run, group, n_q, batch)

Discarded 0.0 annotations.


In [12]:
# load crowd data:
run = '*'
group = 'experiment*'
n_q = '*'
batch = '*'
data_dict_list_crowd = load_experiment_data(run, group, n_q, batch)

Discarded 655.0 annotations.


In [7]:
# clean in preferred way:
run = '*'
group = 'experiment*'
n_q = '*'
batch = '*'

unit = 'pair'
n_stds = 0.5
data_dict_list_crowd_clean = clean_worker_cont_rate(data_dict_list_crowd,\
                                    run,  group,  batch, unit, n_stds)

In [9]:
# aggregate binary labels
expert_bin_labels = aggregate_binary_labels(data_dict_list_exp)
#crowd_bin_labels_raw  = aggregate_binary_labels(data_dict_list_crowd)


In [18]:
units = ['raw', 'pair', 'batch', 'total']
stds = ['raw', 0.5, 1, 1.5, 2]
votes = ['majority_vote', 'top_vote']

overview_dicts = []

for unit in units:
    for n_stdv in stds:
        if unit == str(n_stdv) == 'raw':
            data_dict_list_crowd_clean = data_dict_list_crowd
            crowd_bin_labels_clean  = aggregate_binary_labels(data_dict_list_crowd_clean)
        elif 'raw' not in [unit, n_stdv]:
            data_dict_list_crowd_clean = clean_worker_cont_rate(data_dict_list_crowd,\
                                        run,  group,  batch, unit, n_stdv)
            crowd_bin_labels_clean  = aggregate_binary_labels(data_dict_list_crowd_clean)
            
        else:
            crowd_bin_labels_clean = []
        if crowd_bin_labels_clean != []:
            
            iaa = get_agreement(data_dict_list_crowd_clean, 
                                collapse_relations = False, v=False, disable_kappa = True)
            iaa_levels = get_agreement(data_dict_list_crowd_clean,
                                       collapse_relations = 'levels', v=False, disable_kappa = True)
            for v in votes:
                results_dict = evaluate(expert_bin_labels, crowd_bin_labels_clean, vote=v)
                # write function to get iaa of selected labels only 
                
                d = dict()
                d['cleaning'] = f'cont_rate_{unit}_{n_stdv}'
                d['aggregation'] = v
                d['f1'] = results_dict['f1']
                d['iaa'] = iaa['Krippendorff']
                d['iaa_levels'] = iaa_levels['Krippendorff']
                
                overview_dicts.append(d)

df = pd.DataFrame(overview_dicts)

In [19]:
df.sort_values('f1', ascending = False)

,aggregation,cleaning,f1,iaa,iaa_levels
10,majority_vote,cont_rate_batch_0.5,0.849978,0.359202,0.592076
18,majority_vote,cont_rate_total_0.5,0.820920,0.346002,0.551668
14,majority_vote,cont_rate_batch_1.5,0.811957,0.338154,0.539733
12,majority_vote,cont_rate_batch_1,0.809848,0.346983,0.566461
22,majority_vote,cont_rate_total_1.5,0.808163,0.327822,0.517863
4,majority_vote,cont_rate_pair_1,0.808163,0.353656,0.633026
20,majority_vote,cont_rate_total_1,0.808163,0.334212,0.529893
2,majority_vote,cont_rate_pair_0.5,0.806000,0.365346,0.641770
13,top_vote,cont_rate_batch_1,0.789286,0.346983,0.566461
16,majority_vote,cont_rate_batch_2,0.785093,0.328891,0.531560
